# Преобразование Фурье в изображениях
С помощью этого ноутбука вы на практике познакомитесь с преобразованием Фурье для изображений, а также с его различными способами применения, например:
* Для ускорения применения сверток («Теорема о свертке»)
* Для склейки изображений
* Для поиска фрагмента картинки на полном изображении ([Habr](https://habr.com/ru/post/266129/))

**Перед началом попытайтесь ответить на два вопроса:**
* Для чего применяются преобразования Фурье в математике?
* Зачем нужны преобразования Фурье в изображениях?

_Подробнее про дискретное преобразование Фурье можно прочитать на [Википедии](https://ru.wikipedia.org/wiki/Дискретное_преобразование_Фурье)._

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
from skimage.color import gray2rgb, rgb2gray
from skimage.io import imread

# Increase these if figures appear small
plt.rcParams["figure.figsize"] = fx, fy = (14.08, 6.40)

## Визуализация преобразования Фурье

В данном разделе вам будет предложено реализовать преобразование Фурье для изображений с помощью готовых функций в NumPy, а также проанализировать полученные визуализации.

In [ ]:
!curl -O 'https://courses.cv-gml.ru/storage/seminars/fourier-transform/images.zip'
!unzip -o images.zip

In [ ]:
# Загрузим изображения
imgs = {
    "tiger": imread("images/tiger.jpg"),
    "msu": imread("images/msu.jpg"),
    "cloth": imread("images/cloth.jpg"),
}

In [ ]:
def show_imgs_wrapper(*imgs):
    for img_names, imgset in imgs:
        if not isinstance(imgset, (list, tuple)):
            img_names = (img_names,)
            imgset = (imgset,)

        num = len(imgset)
        cbars = [img.ndim == 2 or img.shape[-1] == 1 for img in imgset]
        num += sum(cbars)
        cbar_width = 0.05
        widths = [r for ax in cbars for r in ([1, cbar_width] if ax else [1])]
        fig, axs = plt.subplots(
            nrows=1,
            ncols=num,
            gridspec_kw={"wspace": 0.01, "hspace": 0.01},
            width_ratios=widths,
            squeeze=True,
            layout="constrained",
        )
        axs = [axs] if num <= 1 else axs

        i, j = 0, 0
        while i < num:
            ax = axs[i]
            i += 1

            img = imgset[j]
            name = img_names[j]
            j += 1

            ax.set_title(name)
            cbar = ax.imshow(img)
            ax.axis("off")

            if img.ndim == 2 or img.shape[-1] == 1:
                ax = axs[i]
                i += 1
                fig.colorbar(cbar, cax=ax)

        plt.show(fig)
        plt.close(fig)

In [ ]:
# Визуализируем изображения
show_imgs_wrapper(*imgs.items())

### Задание: реализуйте функцию визуализации преобразования Фурье для изображения

In [ ]:
def discrete_fourier_transform(img, vis=True):
    # RGB -> Gray
    if img.ndim == 3:
        ...

    # Применим преобразование Фурье из NumPy
    fft = ...

    if vis:
        # Отцентруем
        fft_centered = ...

        # Применим логарифмическое преобразование для наглядности визуализации
        fft_normalized = ...

        return fft_normalized
    else:
        return fft

In [ ]:
imgs_fft = {
    (name, f"{name}_fft"): (image, discrete_fourier_transform(image))
    for name, image in imgs.items()
}
show_imgs_wrapper(*imgs_fft.items())

### Вопросы:
* Чем отличаются друг от друга полученные визуализации преобразования Фурье?
* Какие особенности вы заметили в визуализациях и с чем они связаны?

<br/>
<details>

<br/>
<ul>
    <li>
        <a href="https://en.wikipedia.org/wiki/Nyquist%E2%80%93Shannon_sampling_theorem">Nyquist–Shannon sampling theorem</a>
        (она же - Теорема Котельникова)
    </li>
    <br/>
    <li>
        <a href="https://en.wikipedia.org/wiki/Aliasing">Aliasing</a>
        <img src="https://upload.wikimedia.org/wikipedia/commons/2/28/AliasingSines.svg">
    </li>
</ul>

</details>

## Теорема о свертке

Применение свертки с ядром $K$ к изображению $I$ эквивалентно обратному преобразованию Фурье к поэлементному умножению изображения $I_{Fourier}$ на $K_{Fourier}$, где $I_{Fourier}$ — результат преобразования Фурье для изображения $I$, а $K_{Fourier}$ — результат преобразования Фурье для ядра $K$. _Это и есть «Теорема о свертке»._

**Но в чем же достоинство данной теоремы?**

Пусть изображение $I$ имеет размер $N \times N$, а ядро K — $K \times K$. Тогда, вычисление сверток для всего изображения будет иметь сложность $O(N^2 K^2)$. Если же применить «Теорему о свертке», то сложность составит $O(N^2 log N)$, так как для прямого и обратного преобразований Фурье можно использовать алгоритм [Быстрого_преобразования_Фурье](https://ru.wikipedia.org/wiki/Быстрое_преобразование_Фурье). Таким образом, при большом размере ядра фильтра будем иметь существенный прирост производительности.

In [ ]:
# Загрузим изображение
img = imread("images/cat.jpg", as_gray=True)

In [ ]:
# Визуализируем изображение
show_imgs_wrapper(["cat", gray2rgb(img)])

### Задание:

Напишите функцию применения фильтра Гаусса к изображению с использованием «Теоремы о свертке», для этого:

1. Примените преобразование Фурье к изображению
2. Расширьте нулями и приведите к нужному формату ядро фильтра Гаусса
3. Примените к полученному «раширенному» ядру преобразование Фурье
4. Перемножьте результат применения преобразования Фурье к изображению и к «расширенному» ядру
5. Примените обратное преобразование Фурье

In [ ]:
def pad_kernel(kernel, target):
    th, tw = target
    kh, kw = kernel.shape[:2]
    ph, pw = th - kh, tw - kw

    padding = ...
    kernel = ...

    assert kernel.shape[:2] == target
    return kernel


def conv_theorem(img, kernel):
    """conv(img, kernel) == F^-1 (F(img) * F(kernel))"""

    # Используя discrete_fourier_transform() с vis=False, примените преобразование Фурье к изображению
    img_fourier = ...

    # Расширьте полученное ядро до размера изображения
    kernel_padded = ...

    # Приведите ядро к правильному формату
    kernel_unshifted = ...

    # Используя discrete_fourier_transform() с vis=False, примените преобразование Фурье к «раширенному» ядру
    kernel_fourier = ...

    # Перемножьте результат применения преобразования Фурье к изображению и к «расширенному» ядру
    result = ...

    # Примените обратное преобразование Фурье
    result = ...

    return result

In [ ]:
# Example kernels:
size = 63
identity_kernel = np.zeros((size, size))
identity_kernel[size // 2, size // 2] = 1
# or
unsharp_kernel = np.array(
    [
        [0, -1, 0],
        [-1, 5, -1],
        [0, -1, 0],
    ]
)

In [ ]:
kernel = ...
blurred_theo = ...
blurred_conv = ...

In [ ]:
# Визуализируем результат
F = r"\mathcal{F\,}"
show_imgs_wrapper(
    (["I", "K"], [gray2rgb(img), kernel]),
    (
        [r"I $\ast$ K", rf"${F}^{{-1}}({F}(I) \cdot {F}(K))$"],
        [gray2rgb(blurred_conv.clip(0, 1)), gray2rgb(blurred_theo.clip(0, 1))],
    ),
)

In [ ]:
abs(blurred_conv - blurred_theo).max()

### Задание:
Проанализируйте зависимость скорости работы «стандартного» (с помощью сверток) применения фильтра Гаусса от размера ядра фильтра, аналогичный анализ проведите для случая применения фильтра Гаусса к изображению с использованием «Теоремы о свертке». Также сделайте сравнение графиков для «стандартного» метода и метода, основанного на теореме, о чем можно по ним судить?

In [ ]:
import collections
import time

kernels = []
kernel_sizes = list(range(5, 50, 5))
for kernel_size in kernel_sizes:
    kernels.append(np.random.randn(kernel_size, kernel_size))

results = collections.defaultdict(list)
experiments = {
    "Standard Convolution": signal.convolve2d,
    "Convolution Theorem": conv_theorem,
}
for experiment_name, function in experiments.items():
    for kernel in kernels:
        measurements = []
        for _ in range(10):
            start_time = time.time()
            function(img, kernel)
            end_time = time.time()
            measurements.append(end_time - start_time)

        results[experiment_name].append(sum(measurements) / len(measurements))

In [ ]:
fig, ax = plt.subplots()
ax.set_xlabel("Kernel Size")
ax.set_ylabel("Time (mean)")

for experiment_name, function in experiments.items():
    ax.plot(kernel_sizes, results[experiment_name], label=experiment_name)
ax.legend()
plt.show()

## Фильтр Гаусса и box-фильтр

Данные фильтры применяются для размытия изображений и, как следствие, для уменьшения шума на картинке. Более того, фильтр Гаусса часто используется для предварительной обработки в алгоритмах компьютерного зрения с целью улучшения структуры изображения в различных масштабах.

В данном разделе предлагается визуализировать ядра данных фильтров, а также результаты применения сверток с этими фильтрами для изображений. Но самым важным моментом, который будет затронут в текущем пункте, является визуализация Фурье-образов фильтра Гаусса и box-фильтра. Это поможет вам понять принципы работы рассматриваемых фильтров.

_Подробнее про фильтр Гаусса можно прочитать [здесь](https://ru.wikipedia.org/wiki/Размытие_по_Гауссу), про box-фильтр — [здесь](https://en.wikipedia.org/wiki/Box_blur)._

### Задание: реализуйте функцию, возвращающую ядро фильтра Гаусса

In [ ]:
def gauss_filter_kernel(sigma, kernel_size=63):
    # Используйте signal.windows.gaussian
    gkern1d = ...

    # 1) Separable filters (general case)
    # If there exists such a pair of smaller kernels F1 and F2,
    # such that F_LARGE == conv(F1, F2), then due to associativity
    #     conv(I, F_LARGE) == conv(I, conv(F1, F2)) == conv(conv(I, F1), F2)
    #
    # For the 2d gaussian kernel the following decomposition exists:
    #
    gkern2d = signal.convolve2d(...)

    # 2) In addition, for any pair of kernels F1, F2
    # of shapes (N, 1) and (1, M) the following is true:
    #     conv(F1, F2) = F1 * F2
    # So, in this case, the 2d gaussian kernel can also be expressed as:
    gkern2d = ...

    return gkern2d

In [ ]:
# Визуализация ядра фильтра Гаусса
sigma_vals = (2, 5, 11)
gauss_kernels = [gauss_filter_kernel(sigma) for sigma in sigma_vals]

In [ ]:
show_imgs_wrapper(
    ([f"gauss_filter_kernel({sigma})" for sigma in sigma_vals], gauss_kernels),
)

### Задание: реализуйте функцию, возвращающую ядро box-фильтра

In [ ]:
def box_filter_kernel(kernel_size, pad_until=63):
    kernel = ...

    if pad_until is not None:
        kernel = ...

    return kernel

In [ ]:
# Визуализация ядра box-фильтра
box_vals = (3, 15, 29)
box_kernels = [box_filter_kernel(size) for size in box_vals]

In [ ]:
show_imgs_wrapper(
    ([f"box_filter_kernel({size})" for size in box_vals], box_kernels),
)

### Задание: релизуйте фильтр Гаусса и рассмотрите результаты его применения

In [ ]:
def gauss_blur(img, sigma, kernel_size=63):
    # Используйте conv_theorem
    kernel = gauss_filter_kernel(sigma, kernel_size)
    return conv_theorem(img, kernel)

In [ ]:
gauss_blurs = [img] + [gauss_blur(img, sigma) for sigma in sigma_vals]

In [ ]:
# Визуализируем размытие с помощью фильтра Гаусса
show_imgs_wrapper(
    (
        ["original"] + [f"gauss_blur({sigma})" for sigma in sigma_vals],
        [gray2rgb(I) for I in gauss_blurs],
    ),
    (
        ["original_fft"] + [f"gauss_blur_fft({sigma})" for sigma in sigma_vals],
        [discrete_fourier_transform(I) for I in gauss_blurs],
    ),
)

### Задание: релизуйте box-фильтр и рассмотрите результаты его применения

In [ ]:
def box_blur(img, kernel_size):
    # Используйте conv_theorem
    kernel = box_filter_kernel(kernel_size)
    return conv_theorem(img, kernel)

In [ ]:
box_blurs = [img] + [box_blur(img, size) for size in box_vals]

In [ ]:
# Визуализируем размытие с помощью box-фильтра
show_imgs_wrapper(
    (
        ["original"] + [f"box_blur({sigma})" for sigma in sigma_vals],
        [gray2rgb(I) for I in box_blurs],
    ),
    (
        ["original_fft"] + [f"box_blur_fft({sigma})" for sigma in sigma_vals],
        [discrete_fourier_transform(I) for I in box_blurs],
    ),
)

### Вопросы:
* Какой артефакт возникает при использовании box-фильтра?
* Как этот артефакт можно объяснить с помощью преобразования Фурье?

### Задание: визуализируйте преобразование Фурье для ядер фильтра Гаусса и box-фильтра

In [ ]:
kernels = [gauss_filter_kernel(sigma) for sigma in sigma_vals]
kernel_dfts = [discrete_fourier_transform(kernel, vis=True) for kernel in kernels]

In [ ]:
# Визуализируем преобразование Фурье для фильтра Гаусса
show_imgs_wrapper(
    ([f"gauss_kernel({sigma})" for sigma in sigma_vals], kernels),
    ([f"gauss_kernel({sigma}) -> DFT" for sigma in sigma_vals], kernel_dfts),
)

In [ ]:
kernels = [box_filter_kernel(size) for size in box_vals]
kernel_dfts = [discrete_fourier_transform(kernel, vis=True) for kernel in kernels]

In [ ]:
# Визуализируем преобразование Фурье для box-фильтра
show_imgs_wrapper(
    ([f"box_kernel({size})" for size in box_vals], kernels),
    ([f"box_kernel({size}) -> DFT" for size in box_vals], kernel_dfts),
)

## Склейка изображений
Интересным примером использования фильтра Гаусса для изображений является задача склейки картинок. Ее суть заключается в том, чтобы по маске совместить два изображения и при этом сделать переход от одной картинки к другой максимально плавным. Для этого используются пирамиды Гаусса и Лапласа.

**План действий:**

_Вход_: изображения $A$ и $B$, маска склейки $M$

_Алгоритм_:
1. Строим гауссовские пирамиды $GA$ и $GB$ из изображений $A$ и $B$
2. Строим лапласовские пирамиды $LA$ и $LB$ из пирамид $GA$ и $GB$
3. Строим гауссовскую пирамиду $GM$ из маски $M$
4. «Смешиваем» пирамиды: $LS = GM * LA+ (1−GM) * LB$
5. Строим результат из пирамиды LS

_Более подробно и с поясняющими схемами можно прочитать [здесь](https://courses.cv-gml.ru/storage/seminars/fourier-transform/idar_dyrdal_laplace_blending.pdf) (см. слайды 6-11)._

In [ ]:
# Загрузим изображения
imgs = {
    "a": imread("images/a.png")[..., :3].astype(np.float64) / 255,
    "b": imread("images/b.png")[..., :3].astype(np.float64) / 255,
    "mask": imread("images/mask.jpg", as_gray=True)[..., None],
}

In [ ]:
# Визуализируем изображения
show_imgs_wrapper([list(imgs.keys()), list(imgs.values())])

### Задание: напрямую смешайте два изображения по маске

In [ ]:
mask = ...
a = ...
b = ...
masked_a = ...
masked_b = ...
naive_blend = ...

In [ ]:
# Визуализируем результат
show_imgs_wrapper(
    (
        ["M", "(1-M) * a", "M * b", "naive_blend"],
        [mask, masked_a, masked_b, naive_blend],
    ),
)

### Задание: смешайте два изображения по сглаженной маске

In [ ]:
from skimage.exposure import adjust_gamma
from skimage.filters import gaussian

gamma = 2.2

In [ ]:
smooth_mask = ...
a = ...
b = ...
masked_a = ...
masked_b = ...
smooth_blend = ...

In [ ]:
# Визуализируем результат
show_imgs_wrapper(
    (
        ["M", "(1-M) * a", "M * b", "smooth_blend"],
        [smooth_mask, masked_a, masked_b, smooth_blend],
    ),
)

### Задание: реализовать пирамиды Гаусса и Лапласа, а также функции «смешивания» пирамид и склейки изображений

In [ ]:
from skimage.transform import resize

In [ ]:
n_layers = 4
base_mask = gaussian(imgs["mask"], 4)
a = adjust_gamma(imgs["a"], gamma)
b = adjust_gamma(imgs["b"], gamma)

In [ ]:
def gaussian_pyramid(img, n_layers, sigma):
    """Построение пирамиды Гаусса

    Вход: изображение, количество уровней, sigma
    Выход: уровни пирамиды

    """
    pyramid = ...
    return pyramid

In [ ]:
gaussian_mask = gaussian_pyramid(base_mask, n_layers, 16)

In [ ]:
show_imgs_wrapper(
    ([f"gaussian_mask[{i}]" for i in range(n_layers)], gaussian_mask),
)

In [ ]:
gaussian_a = gaussian_pyramid(a, n_layers, 16)

In [ ]:
show_imgs_wrapper(
    (
        [f"gaussian_a[{i}]" for i in range(n_layers)],
        [adjust_gamma(i, 1 / gamma) for i in gaussian_a],
    ),
)

In [ ]:
def laplacian_pyramid(img, n_layers, sigma):
    """Построение пирамиды Лапласа

    Вход: изображение, количество уровней, sigma
    Выход: уровни пирамиды

    """
    pyramid_gauss = gaussian_pyramid(img, n_layers, sigma)
    pyramid = ...
    return pyramid

In [ ]:
laplacian_a = laplacian_pyramid(a, n_layers, 16)
laplacian_b = laplacian_pyramid(b, n_layers, 16)

In [ ]:
show_imgs_wrapper(
    ([f"laplacian_a[{i}]" for i in range(n_layers)], [abs(img) for img in laplacian_a]),
    ([f"laplacian_b[{i}]" for i in range(n_layers)], [abs(img) for img in laplacian_b]),
)

In [ ]:
def blend_pyramids(pyramid_a, pyramid_b, pyramid_mask):
    """Смешивание пирамид

    Вход: пирамида Лапласа для первого иззображения,
          пирамида Лапласа для второго изображения,
          пирамида Гаусса для маски
    Выход: «смешанная» пирамида

    """
    blend_pyramid = ...
    return blend_pyramid

In [ ]:
result_pyramid = blend_pyramids(laplacian_a, laplacian_b, gaussian_mask)

In [ ]:
result_pyramid_abs = [abs(img) for img in result_pyramid]
show_imgs_wrapper(
    ([f"gaussian_mask[{i}]" for i in range(n_layers)], gaussian_mask),
    ([f"result_pyramid[{i}]" for i in range(n_layers)], result_pyramid_abs),
)

In [ ]:
def blend_image(blend_pyramid):
    """Склейка изображений

    Вход: «смешанная» пирамида
    Выход: склеенное изображение

    """
    target_shape = blend_pyramid[0].shape
    img = ...
    return img

In [ ]:
pyramid_blend = blend_image(result_pyramid).clip(0, 1)
pyramid_blend = adjust_gamma(pyramid_blend, 1 / gamma)

In [ ]:
show_imgs_wrapper(
    (
        ["naive", "smooth", "pyramid"],
        [naive_blend, smooth_blend, pyramid_blend],
    ),
)